<a href="https://colab.research.google.com/github/ryota-sb/py-colaboratory/blob/main/pianoroll_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.6) ...
     |████████████████████████████████| 5.6 MB 9.8 MB/s 
     |████████████████████████████████| 51 kB 7.6 MB/s 
     |████████████████████████████████| 1.4 MB 7.4 MB/s 
     |████████████████████████████████| 352 kB 63.7 MB/s 
     |████████████████████████████████| 210 kB 56.3 MB/s 
     |████████████████████████████████| 3.4 MB 39.8 MB/s 
     |████████████████████████████████| 1.6 MB 54.9 MB/s 
     |████████████████████████████████| 3.1 MB

In [4]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

seed = music_pb2.NoteSequence()

seed.notes.add(pitch=80, start_time=0.0, end_time=0.4, velocity=80)
seed.notes.add(pitch=80, start_time=0.4, end_time=0.8, velocity=80)
seed.notes.add(pitch=87, start_time=0.8, end_time=1.2, velocity=80)
seed.notes.add(pitch=87, start_time=1.2, end_time=1.6, velocity=80)
seed.notes.add(pitch=89, start_time=1.6, end_time=2.0, velocity=80)
seed.notes.add(pitch=89, start_time=2.0, end_time=2.4, velocity=80)

seed.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
seed.notes.add(pitch=62, start_time=0.4, end_time=0.8, velocity=80)
seed.notes.add(pitch=64, start_time=0.8, end_time=1.2, velocity=80)
seed.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
seed.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
seed.notes.add(pitch=64, start_time=2.0, end_time=2.4, velocity=80)

seed.notes.add(pitch=49, start_time=0.0, end_time=0.8, velocity=80)
seed.notes.add(pitch=44, start_time=0.8, end_time=1.6, velocity=80)
seed.notes.add(pitch=40, start_time=1.6, end_time=2.4, velocity=80)

seed.total_time = 2.4
seed.tempos.add(qpm=75)

note_seq.plot_sequence(seed)
note_seq.play_sequence(seed, synth=note_seq.fluidsynth)

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

In [5]:
from magenta.models.pianoroll_rnn_nade import pianoroll_rnn_nade_sequence_generator
from magenta.models.shared import sequence_generator_bundle

note_seq.notebook_utils.download_bundle("pianoroll_rnn_nade.mag", "/models/")
bundle = sequence_generator_bundle.read_bundle_file("/models/pianoroll_rnn_nade.mag")
generator_map = pianoroll_rnn_nade_sequence_generator.get_generator_map()
pianoroll_rnn_nade = generator_map["rnn-nade_attn"](checkpoint=None, bundle=bundle)
pianoroll_rnn_nade.initialize()

In [7]:
from note_seq.protobuf import generator_pb2

total_time = 180
temperature = 1.0

base_end_time = max(note.end_time for note in seed.notes)

generator_options = generator_pb2.GeneratorOptions()
generator_options.args["temperature"].float_value = temperature
generator_options.generate_sections.add(
    start_time=base_end_time,
    end_time=total_time)

gen_seq = pianoroll_rnn_nade.generate(seed, generator_options)

note_seq.plot_sequence(gen_seq)
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)